# **RD2022 Data Processing**
 Below is the code that was used to process the RD 2022 dataset mentioned in the proposal. The general process requires sorting data into pothole and no pothole examples, downsizing all images to 512x512 resolution, and spliting the data into testing, validation, and training. More information on the dataset, its statistics, and the preprocessing procedure can be found in the progress report under section 3.

# Mount Google Drive and Data

In [ ]:
import os
import json
import shutil
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Example of JSON File


In [ ]:
'''
{
    "description": "",
    "tags": [
        {
            "id": 16583603,
            "tagId": 29662,
            "name": "India",
            "value": null,
            "labelerLogin": "inbox@datasetninja.com",
            "createdAt": "2024-01-22T16:16:46.680Z",
            "updatedAt": "2024-01-22T16:16:46.680Z"
        }
    ],
    "size": {
        "height": 720,
        "width": 720
    },
    "objects": [
        {
            "id": 138998396,
            "classId": 6512002,
            "objectId": null,
            "description": "",
            "geometryType": "rectangle",
            "labelerLogin": "inbox@datasetninja.com",
            "createdAt": "2024-01-22T16:16:46.690Z",
            "updatedAt": "2024-01-22T16:16:46.690Z",
            "tags": [
                {
                    "id": 104985322,
                    "tagId": 29666,
                    "name": "detail",
                    "value": "rutting, bump, pothole, separation",
                    "labelerLogin": "inbox@datasetninja.com",
                    "createdAt": "2024-01-22T16:16:46.690Z",
                    "updatedAt": "2024-01-22T16:16:46.690Z"
                }
            ],
            "classTitle": "pothole",
            "points": {
                "exterior": [
                    [
                        358,
                        422
                    ],
                    [
                        463,
                        455
                    ]
                ],
                "interior": []
            }
        },
        {
            "id": 138998395,
            "classId": 6512002,
            "objectId": null,
            "description": "",
            "geometryType": "rectangle",
            "labelerLogin": "inbox@datasetninja.com",
            "createdAt": "2024-01-22T16:16:46.687Z",
            "updatedAt": "2024-01-22T16:16:46.687Z",
            "tags": [
                {
                    "id": 104985321,
                    "tagId": 29666,
                    "name": "detail",
                    "value": "rutting, bump, pothole, separation",
                    "labelerLogin": "inbox@datasetninja.com",
                    "createdAt": "2024-01-22T16:16:46.687Z",
                    "updatedAt": "2024-01-22T16:16:46.687Z"
                }
            ],
            "classTitle": "pothole",
            "points": {
                "exterior": [
                    [
                        1,
                        551
                    ],
                    [
                        43,
                        610
                    ]
                ],
                "interior": []
            }
        },
        {
            "id": 138998394,
            "classId": 6512001,
            "objectId": null,
            "description": "",
            "geometryType": "rectangle",
            "labelerLogin": "inbox@datasetninja.com",
            "createdAt": "2024-01-22T16:16:46.686Z",
            "updatedAt": "2024-01-22T16:16:46.686Z",
            "tags": [
                {
                    "id": 104985320,
                    "tagId": 29666,
                    "name": "detail",
                    "value": "partial pavement, overall pavement",
                    "labelerLogin": "inbox@datasetninja.com",
                    "createdAt": "2024-01-22T16:16:46.685Z",
                    "updatedAt": "2024-01-22T16:16:46.685Z"
                }
            ],
            "classTitle": "alligator crack",
            "points": {
                "exterior": [
                    [
                        21,
                        495
                    ],
                    [
                        347,
                        685
                    ]
                ],
                "interior": []
            }
        },
        {
            "id": 138998393,
            "classId": 6512003,
            "objectId": null,
            "description": "",
            "geometryType": "rectangle",
            "labelerLogin": "inbox@datasetninja.com",
            "createdAt": "2024-01-22T16:16:46.681Z",
            "updatedAt": "2024-01-22T16:16:46.681Z",
            "tags": [
                {
                    "id": 104985319,
                    "tagId": 29666,
                    "name": "detail",
                    "value": "white line blur",
                    "labelerLogin": "inbox@datasetninja.com",
                    "createdAt": "2024-01-22T16:16:46.681Z",
                    "updatedAt": "2024-01-22T16:16:46.681Z"
                }
            ],
            "classTitle": "other corruption",
            "points": {
                "exterior": [
                    [
                        322,
                        485
                    ],
                    [
                        379,
                        549
                    ]
                ],
                "interior": []
            }
        }
    ]
}
'''

'\n{\n    "description": "",\n    "tags": [\n        {\n            "id": 16583603,\n            "tagId": 29662,\n            "name": "India",\n            "value": null,\n            "labelerLogin": "inbox@datasetninja.com",\n            "createdAt": "2024-01-22T16:16:46.680Z",\n            "updatedAt": "2024-01-22T16:16:46.680Z"\n        }\n    ],\n    "size": {\n        "height": 720,\n        "width": 720\n    },\n    "objects": [\n        {\n            "id": 138998396,\n            "classId": 6512002,\n            "objectId": null,\n            "description": "",\n            "geometryType": "rectangle",\n            "labelerLogin": "inbox@datasetninja.com",\n            "createdAt": "2024-01-22T16:16:46.690Z",\n            "updatedAt": "2024-01-22T16:16:46.690Z",\n            "tags": [\n                {\n                    "id": 104985322,\n                    "tagId": 29666,\n                    "name": "detail",\n                    "value": "rutting, bump, pothole, separati

# Deleting Folders if Needed


This was used for initial testing with the session storage rather than Google Drive.

In [ ]:
#shutil.rmtree('/content/with_potholes')
#shutil.rmtree('/content/without_potholes')
#shutil.rmtree('/content/train_jsons')


# Divide Data into Positive and Negative Folders

In [ ]:
def load_image_and_annotations(img_path, json_path):
    image = cv2.imread(img_path)

    with open(json_path, 'r') as f:
        data = json.load(f)

    bboxes = []
    for obj in data.get("objects", []):
        if obj.get("classTitle") == "pothole":
            exterior = obj["points"]["exterior"]
            if len(exterior) == 2:
                (x_min, y_min), (x_max, y_max) = exterior
                bboxes.append([x_min, y_min, x_max, y_max])

    return image, bboxes


In [ ]:
def resize_image_and_bboxes(image, bboxes, output_size=512, original_size=2760):
  if original_size == output_size:
    return image, bboxes

  resized_image = cv2.resize(image, (output_size, output_size))
  scale = output_size / original_size
  resized_bboxes = []

  for x_min, y_min, x_max, y_max in bboxes:
    resized_bboxes.append([
        int(x_min * scale),
        int(y_min * scale),
        int(x_max * scale),
        int(y_max * scale)
    ])

  return resized_image, resized_bboxes

In [ ]:
import os
import json
import cv2
import shutil
import torch

# Set correct path to json folder with labels
''' raw_json_dir = "/content/test_jsons/ann"
raw_image_folder = "/content/raw_images/img" '''

raw_json_dir = "/content/drive/My Drive/APS360_Project/RDD_2022_Data/train/ann"
raw_image_folder = "/content/drive/My Drive/APS360_Project/RDD_2022_Data/train/img"

with_potholes_json_dir = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/with_potholes_json"

with_potholes_image_dir = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/with_potholes_images"
without_potholes_image_dir = "/content/drive/ MyDrive/APS360_Project/RDD_2022_ProcessedData/without_potholes_images"

potholeExamples = 0
noPotholeExamples = 0
totalExamples = 0

all_examples = [f for f in os.listdir(raw_json_dir) if os.path.isfile(os.path.join(raw_json_dir, f))]
num_all_examples = len(all_examples)

# Look through each json file in the folder
for json_name in os.listdir(raw_json_dir):

    # get the image path (remove the .json at the end of the name)
    imageName = json_name[0:-5]
    image_path = os.path.join(raw_image_folder, imageName)

    json_path = os.path.join(raw_json_dir, json_name)
    img, bbox = load_image_and_annotations(image_path, json_path)


    if json_name.endswith(".json"):
        path = os.path.join(raw_json_dir, json_name)
        with open(path, 'r') as f:
            try:

                #================= 1) if the image is Norway, skip it =================
                labels = json.load(f)
                tags = labels.get("tags", [])
                if any("Norway" in tag.get("name", "") for tag in tags):
                    print(f"Skipping {json_name}")
                    continue
                else:
                    print(f"Processing {json_name}")

                #================= 2) Resize images =================
                if 'size' in labels:
                    width = labels['size'].get('width')
                    height = labels['size'].get('height')
                    if width != height:
                      print(f"ERROR: {json_name} is not a square image!")
                    else:
                      resized_img, resized_bboxes = resize_image_and_bboxes(img, bbox, 512, width)
                    ''' else:
                      print(f"{json_name} → Width: {width}, Height: {height}") '''
                else:
                    print(f"{json_name} size is missing!")


                #================= 3) Split data into potholes and no potholes =================
                objects = labels.get("objects", [])
                image_has_pothole = any(obj.get("classTitle", "").lower() == "pothole" for obj in objects)
                # save json and jpeg into its corresponding folder

                # Create a new json and write the labels for potholes
                if image_has_pothole:
                    if not os.path.exists( os.path.join(with_potholes_json_dir, json_name)):
                      ann_data = {
                          "size": {
                              "height": 512,
                              "width": 512
                          },
                          "objects":[
                              {
                                  "geometryType": "rectangle",
                                  "points": {
                                      "exterior": [
                                          [x_min, y_min],
                                          [x_max, y_max]
                                      ],
                                      "interior": []
                                  },
                                  "classTitle": "pothole",
                              }
                              for x_min, y_min, x_max, y_max in resized_bboxes
                          ]
                      }
                      out_json_name = os.path.join(with_potholes_json_dir, json_name)
                      with open(out_json_name, 'w') as f:
                          json.dump(ann_data, f, indent=2)


                    if not os.path.exists( os.path.join(with_potholes_image_dir, imageName)):
                      cv2.imwrite(os.path.join(with_potholes_image_dir, imageName), resized_img)

                    potholeExamples += 1

                # No pothole examples
                else:
                    noPotholeExamples += 1

                totalExamples += 1
                print(f"Completed {totalExamples} out of {num_all_examples}")

            # error if JSON file isnt found
            except json.JSONDecodeError:
                print(f"{json_name} JSON not found")

print(f"Found {potholeExamples} pothole examples and {noPotholeExamples} no pothole examples")
print(f"Found {totalExamples} total examples")

Streaming output truncated to the last 5000 lines.
Processing [[267, 527, 309, 559]]
Processing Japan_001802.jpg.json
Completed 28559 out of 38385
Processing []
Processing Japan_009852.jpg.json
Completed 28560 out of 38385
Processing []
Processing Japan_004148.jpg.json
Completed 28561 out of 38385
Processing []
Processing Japan_010196.jpg.json
Completed 28562 out of 38385
Processing []
Processing Japan_001673.jpg.json
Completed 28563 out of 38385
Processing [[391, 436, 430, 459], [469, 388, 517, 407]]
Processing Japan_010963.jpg.json
Completed 28564 out of 38385
Processing []
Processing Japan_010043.jpg.json
Completed 28565 out of 38385
Processing []
Processing Japan_005296.jpg.json
Completed 28566 out of 38385
Processing []
Processing Japan_012798.jpg.json
Completed 28567 out of 38385
Processing []
Processing Japan_010159.jpg.json
Completed 28568 out of 38385
Processing []
Processing Japan_005549.jpg.json
Completed 28569 out of 38385
Processing []
Processing Japan_007163.jpg.json
Comp

# Resolving Class Imbalance

Since there are signficantly more examples without potholes than without, we will need to balance our data. This can be done by randomly removing examples of the no potholes group until the two groups are balanced.

If we require more data in the future, we can take a different approach and augment the pothole data to achieve a balance too.

In [ ]:
import os
import random
seed = 5

with_potholes_json_dir = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/with_potholes_json"
without_potholes_json_dir = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/without_potholes_json"

with_potholes_image_dir = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/with_potholes_images"
without_potholes_image_dir = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/without_potholes_images"

potholeExamples = len([f for f in os.listdir(with_potholes_json_dir) if os.path.isfile(os.path.join(with_potholes_json_dir, f))])
noPotholeExamples = len([f for f in os.listdir(without_potholes_json_dir) if os.path.isfile(os.path.join(without_potholes_json_dir, f))])
# Find which data set is imbalanced
if potholeExamples > noPotholeExamples:
  print("With potholes is imbalanced")
  json_dir = with_potholes_json_dir
  image_dir = with_potholes_image_dir
  num_samples_to_delete = potholeExamples - noPotholeExamples
  print(f"Deleting {num_samples_to_delete} samples")
  all_json_files = [f for f in os.listdir(with_potholes_json_dir) if os.path.isfile(os.path.join(with_potholes_json_dir, f))]
  all_img_files = [f for f in os.listdir(with_potholes_image_dir) if os.path.isfile(os.path.join(with_potholes_image_dir, f))]

else:
  print("Without potholes is imbalanced")
  json_dir = without_potholes_json_dir
  image_dir = without_potholes_image_dir
  num_samples_to_delete = noPotholeExamples - potholeExamples
  print(f"Deleting {num_samples_to_delete} samples")
  all_json_files = [f for f in os.listdir(without_potholes_json_dir) if os.path.isfile(os.path.join(without_potholes_json_dir, f))]
  print(all_json_files)
  all_img_files = [f for f in os.listdir(without_potholes_image_dir) if os.path.isfile(os.path.join(without_potholes_image_dir, f))]

# Randomly select data to delete
random.seed(seed)
json_files_to_delete = random.sample(all_json_files, num_samples_to_delete)
random.seed(seed)
img_files_to_delete = random.sample(all_img_files, num_samples_to_delete)

# Delete all the data
total_json_deleted = 0
total_img_deleted = 0
for data in json_files_to_delete:
  os.remove(os.path.join(json_dir, data))
  total_json_deleted += 1
  print(f"Deleted {total_json_deleted} json files")
for data in img_files_to_delete:
  os.remove(os.path.join(image_dir, data))
  total_img_deleted += 1
  print(f"Deleted {total_img_deleted} image files")

print(f"Deleted {total_json_deleted} json files and {total_img_deleted} image files")





Streaming output truncated to the last 5000 lines.
Deleted 260 json files
Deleted 261 json files
Deleted 262 json files
Deleted 263 json files
Deleted 264 json files
Deleted 265 json files
Deleted 266 json files
Deleted 267 json files
Deleted 268 json files
Deleted 269 json files
Deleted 270 json files
Deleted 271 json files
Deleted 272 json files
Deleted 273 json files
Deleted 274 json files
Deleted 275 json files
Deleted 276 json files
Deleted 277 json files
Deleted 278 json files
Deleted 279 json files
Deleted 280 json files
Deleted 281 json files
Deleted 282 json files
Deleted 283 json files
Deleted 284 json files
Deleted 285 json files
Deleted 286 json files
Deleted 287 json files
Deleted 288 json files
Deleted 289 json files
Deleted 290 json files
Deleted 291 json files
Deleted 292 json files
Deleted 293 json files
Deleted 294 json files
Deleted 295 json files
Deleted 296 json files
Deleted 297 json files
Deleted 298 json files
Deleted 299 json files
Deleted 300 json files
Delete

## Splitting Data


Below are helper functions for copying and randomizing the data.

In [ ]:
def copy_files(file_list, to_folder, from_folder):
    total_items = len(file_list)
    items_copied = 0
    for f in file_list:
      if not os.path.exists( os.path.join(to_folder, f) ):
        shutil.copy(os.path.join(from_folder, f), os.path.join(to_folder, f))
        items_copied += 1
        print(f"Copied {items_copied} out of {total_items}")

In [ ]:
import random
seed = 5

def get_shuffle_data(source_folder):
  all_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
  random.seed(seed)
  random.shuffle(all_files)
  return all_files


In [ ]:
def copy_data_split(all_files, training_split, validation_split, testing_split, training_folder, validation_folder, testing_folder, from_folder):
  training_files = all_files[:training_split]
  validation_files = all_files[training_split:training_split + validation_split]
  testing_files = all_files[training_split + validation_split:training_split + validation_split + testing_split]

  print(f"Copying training files...")
  copy_files(training_files, training_folder, from_folder)
  print(f"Copying validation files...")
  copy_files(validation_files, validation_folder, from_folder)
  print(f"Copying testing files...")
  copy_files(testing_files, testing_folder, from_folder)

Defines and creates folders for the training, validation, and testing split.

In [ ]:
training_json_folder = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/train/json"
training_img_folder = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/train/img"
os.makedirs(training_json_folder, exist_ok=True)
os.makedirs(training_img_folder, exist_ok=True)

validation_json_folder = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/valid/json"
validation_img_folder = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/valid/img"
os.makedirs(validation_json_folder, exist_ok=True)
os.makedirs(validation_img_folder, exist_ok=True)

testing_json_folder = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/test/json"
testing_img_folder = "/content/drive/My Drive/APS360_Project/RDD_2022_ProcessedData/test/img"
os.makedirs(testing_json_folder, exist_ok=True)
os.makedirs(testing_img_folder, exist_ok=True)

In [ ]:
# Split ratio
training_ratio = 0.7
valid_ratio = 0.15
testing_ratio = 0.15

# Gather all the files from the 4 folders
all_pothole_json_files = get_shuffle_data(with_potholes_json_dir)
#all_no_pothole_json_files = get_shuffle_data(without_potholes_json_dir)
all_pothole_image_files = get_shuffle_data(with_potholes_image_dir)
all_no_pothole_image_files = get_shuffle_data(without_potholes_image_dir)

# The number of data in each folder should be the same
total = len(all_pothole_json_files)
training_split = int(training_ratio * total)
validation_split = int(valid_ratio * total)
testing_split = int(testing_ratio * total)
print(f"Training split: {training_split}, Validation split: {validation_split}, Testing split: {testing_split}")

print(f"Copying all_pothole_json_files...")
copy_data_split(all_pothole_json_files, training_split, validation_split, testing_split, training_json_folder, validation_json_folder, testing_json_folder, with_potholes_json_dir)
print(f"Copying all_no_pothole_json_files...")
#copy_data_split(all_no_pothole_json_files, training_split, validation_split, testing_split, training_json_folder, validation_json_folder, testing_json_folder, without_potholes_json_dir)

print(f"Copying all_pothole_image_files...")
copy_data_split(all_pothole_image_files, training_split, validation_split, testing_split, training_img_folder, validation_img_folder, testing_img_folder, with_potholes_image_dir)
print(f"Copying all_no_pothole_image_files...")
copy_data_split(all_no_pothole_image_files, training_split, validation_split, testing_split, training_img_folder, validation_img_folder, testing_img_folder, without_potholes_image_dir)

Streaming output truncated to the last 5000 lines.
Copied 103 out of 355
Copied 104 out of 355
Copied 105 out of 355
Copied 106 out of 355
Copied 107 out of 355
Copied 108 out of 355
Copied 109 out of 355
Copied 110 out of 355
Copied 111 out of 355
Copied 112 out of 355
Copied 113 out of 355
Copied 114 out of 355
Copied 115 out of 355
Copied 116 out of 355
Copied 117 out of 355
Copied 118 out of 355
Copied 119 out of 355
Copied 120 out of 355
Copied 121 out of 355
Copied 122 out of 355
Copied 123 out of 355
Copied 124 out of 355
Copied 125 out of 355
Copied 126 out of 355
Copied 127 out of 355
Copied 128 out of 355
Copied 129 out of 355
Copied 130 out of 355
Copied 131 out of 355
Copied 132 out of 355
Copied 133 out of 355
Copied 134 out of 355
Copied 135 out of 355
Copied 136 out of 355
Copied 137 out of 355
Copied 138 out of 355
Copied 139 out of 355
Copied 140 out of 355
Copied 141 out of 355
Copied 142 out of 355
Copied 143 out of 355
Copied 144 out of 355
Copied 145 out of 355
Cop